<a href="https://colab.research.google.com/github/Nami-59/VK/blob/main/VK_friends_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vk_api

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import vk_api

In [3]:
vkApiSession= vk_api.VkApi(token="vk1.a.2Li16wh621wnMMmkrquuvMl2z81eO2War6issca0LuZIsKzYA3sr2IzQDUoyYCwIlELsDsdM19oFpKP-_lR68FwXzJFMJ9bjifm-93Us5vspR118RDMaA9mWdWVaBAEJfENGwFD6oV6MmppyOSS-2EzNwUtpZKlrgX5Vh3h6mRqWdD5dcpP7o_4KgWQQcB53QqDqNer5plY905UQFfBcNg")
vk = vkApiSession.get_api()

In [ ]:
def friends(id):
  try:
    m=vk.friends.get(user_id = str(id), count = 50000)
    for q in m['items']:
       with open('friends.txt','a') as f2:
           f2.write(str(id) + ';' + str(q) + '\n')
  except:
    with open('friends.txt','a') as f2:
        f2.write(str(id) + ';0\n')

In [ ]:
my_id = pd.read_csv('id_group.txt', header=None) #первый слой

In [ ]:
for i in range(0, len(my_id)):
  friends(my_id[0][i]) #второй слой

In [ ]:
data = pd.read_csv('friends.txt', sep=';',header=None)

In [ ]:
data1 = data[1].unique()
data1 = [x for x in data1 if x not in set(my_id)]

In [ ]:
for j in range(0, len(data1)):
  friends(data1[j]) #третий слой

In [ ]:
data_1 = pd.read_csv('friends.txt', sep=';',header=None)
for j in range(0, len(data_1)):
  if data_1[1][j] == 0:
    data_1 = data_1.drop(j)

In [ ]:
data2 = data_1[1].unique()
data_2 = data_1[0].unique()
data2 = [x for x in data2 if x not in set(data_2)]


In [ ]:
with open('third_layer.txt','a') as f2:
    f2.write(str(data2) + '\n')

In [ ]:
def get_indices(lst, el):
    t = list()
    for i in range(0, len(lst)):
        if lst[i] == el:
            t.append(i)
    return t

In [ ]:
def are_friends(df, i):
  k = list()
  v = 0
  id = list()
  #for j in range(0, 2000, 1000):
  for j in range(0,780000,1000):
    v+=1000
    m=vk.friends.areFriends(user_ids = [df[i], df[j:v]])
    for w in range(0,len(m)):
      k.append(m[w]['friend_status'])
      id.append(m[w]['user_id'])
  return k, id

In [ ]:
def are_friends_78(df, i):
  k = list()
  id = list()
  #m=vk.friends.areFriends(user_ids = [df[i], df[2000:len(df)]])
  m=vk.friends.areFriends(user_ids = [df[i], df[781000:len(df)]])
  for j in range(0,len(m)):
    k.append(m[j]['friend_status'])
    id.append(m[j]['user_id'])
  return k, id

In [ ]:
def friends_friends(df, j_index):
  for j in range(j_index, len(df)-1):
    print(j)
    k, id = are_friends(df, j)
    k1, id1 = are_friends_78(df, j)
    k+=k1
    id+=id1
    i = get_indices(k, 3)
    for v in range(0,len(i)):
      with open('friends_dop.txt','a') as f2:
        f2.write(str(df[j]) + ';' + str(id[i[v]]) + '\n')

In [ ]:
friends_friends(data2, 0)

Центральности

In [4]:
import networkx as nx

In [48]:
data = pd.read_csv('friends.txt', sep=';',header=None)
data_3 = pd.read_csv('friends_dop.txt', sep=';',header=None)
df = data.append(data_3, ignore_index = True)

<ipython-input-48-2ce531aadc7f>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = data.append(data_3, ignore_index = True)


In [49]:
for j in range(0, len(df)):
  if df[1][j] == 0:
    df = df.drop(j)

In [50]:
G = nx.Graph()
nodes = df[1].unique()
edges = df.values
G.add_nodes_from(nodes)
G.add_edges_from(edges)
#len(nodes), len(edges)

In [51]:
my_id = pd.read_csv('id_group.txt', header=None)

In [52]:
def centr(df, id):
  df1 = list(df.items())
  line = list()
  id_group = list()
  for i in range(0, len(id)):
    for j in range(0, len(df1)):
      if id[0][i] == df1[j][0]:
          id_group.append(my_id[0][i])
          line.append(df1[j][1])
          break
  return id_group, line

In [53]:
close_centr = nx.closeness_centrality(G)

In [54]:
id_group_close, line_close = centr(close_centr, my_id)

In [55]:
between_centr = nx.betweenness_centrality(G)

In [56]:
id_group_between, line_between = centr(between_centr, my_id)

In [57]:
eigen_centr = nx.eigenvector_centrality(G, max_iter = 1000)

In [58]:
id_group_eigen, line_eigen = centr(eigen_centr, my_id)

In [59]:
table = pd.DataFrame({
    # 'id_group_close': id_group_close,
     'closeness_centrality': line_close,
    # 'id_group_between': id_group_between,
     'betweenness_centrality': line_between,
    # 'id_group_eigen': id_group_eigen,
     'eigenvector_centrality': line_eigen
}, index=id_group_close)
table.index.name = 'id'
table.to_csv('centrality.csv')

In [60]:
first_name = list()
for i in range(0,len(id_group_close)):
  user_get = vk.users.get(user_ids=(id_group_close[i]))
  first_name.append(user_get[0]['first_name'])

In [63]:
table.index = first_name
table.index.name = 'First name'
table

,closeness_centrality,betweenness_centrality,eigenvector_centrality
First name,,,
Артём,0.356485,0.000149,0.013098
Дмитрий,0.381672,0.017644,0.140279
Игорь,0.393819,0.008854,0.094531
Мария,0.343469,0.000478,0.020047
Александр,0.350741,0.000470,0.020413
Tr,0.364103,0.000064,0.009434
Яна,0.375661,0.001517,0.028390
Алексей,0.502613,0.000085,0.020303
Владислав,0.342798,0.000793,0.026102


In [64]:
#Центры в группе
max_centr = pd.DataFrame({
     'First name': table.idxmax(),
     'Значение': table.max(),
})
max_centr

,First name,Значение
closeness_centrality,Алексей,0.502613
betweenness_centrality,Дмитрий,0.017644
eigenvector_centrality,Дмитрий,0.140279


In [65]:
cl = table['closeness_centrality'].sort_values(ascending=False)
bet = table['betweenness_centrality'].sort_values(ascending=False)
eig = table['eigenvector_centrality'].sort_values(ascending=False)

In [66]:
index_max = pd.DataFrame({
     'closeness_centrality_max': cl.index,
     'betweenness_centrality_max': bet.index,
     'eigenvector_centrality_max': eig.index
})
index_max

,closeness_centrality_max,betweenness_centrality_max,eigenvector_centrality_max
0,Алексей,Дмитрий,Дмитрий
1,Елисей,Игорь,Игорь
2,Тимофей,Енот,Енот
3,Игорь,Владислав,Владислав
4,Дмитрий,Виктория,Виктория
5,Илья,Кирилл,Кирилл
6,Яна,Яна,Виталий
7,Кирилл,Виталий,Иван
8,Алексей,Иван,Яна
9,Максим,Владислав,Алексей
